In [2]:
import os
import zipfile
from datetime import datetime, timedelta
import subprocess

In [3]:
# SFTP Credentials
# host = 'ec-pdgs-dissemination1.eo.esa.int'
host       = 'ec-pdgs-dissemination2.eo.esa.int'
username   = 'arthur.avenas@polytechnique.org'
# password   = '??'

start_date = datetime(2025, 2, 5)
end_date   = datetime(2025, 2, 28)

products_baselines = {
    # 'ATL_NOM_1B': ['AD'],
    'AC__TC__2B': ['AB'],
    # 'MSI_COP_2A': ['AB'],
    # 'CPR_FMR_2A': ['AB']
}

In [ ]:
# Bulk download

lftp_script = f"""
open -u {username},{password} ftps://{host}
set ssl:verify-certificate no
set ftp:ssl-auth TLS
set ftp:passive-mode true
"""

current_date = start_date
while current_date <= end_date:
    year  = str(current_date.year)
    month = f"{current_date.month:02d}"
    day   = f"{current_date.day:02d}"

    for product_name, baselines in products_baselines.items():
        local_dir = f'/home/arthur/Téléchargements/test/{year}{month}{day}/{product_name}/'

        # Ensure the local directory exists
        os.makedirs(local_dir, exist_ok=True)
        print(f"Created local directory: {local_dir}")

        for baseline in baselines:
            remote_dir = f'EarthCARE/EarthCAREL1Validated/{product_name}/{baseline}/{year}/{month}/{day}'
            # remote_dir = f'EarthCARE/EarthCAREL2Products/{product_name}/{baseline}/{year}/{month}/{day}'

            # Append the mirror command to the lftp script
            lftp_script += f"""
            lcd {local_dir}
            mirror --verbose {remote_dir} .
            """

    current_date += timedelta(days=1)

lftp_script += "quit\n"

# Execute lftp command
try:
    process = subprocess.run(
        ["lftp", "-c", lftp_script],
        capture_output=True,
        text=True
    )

    print("STDOUT:", process.stdout)
    print("STDERR:", process.stderr)

    if process.returncode == 0:
        print("FTPS file transfer completed successfully")
    else:
        print("FTPS transfer failed")

except Exception as e:
    print("Error executing lftp command:", e)

Created local directory: /home/arthur/Téléchargements/test/20250205/AC__TC__2B/
Created local directory: /home/arthur/Téléchargements/test/20250206/AC__TC__2B/
Created local directory: /home/arthur/Téléchargements/test/20250207/AC__TC__2B/
Created local directory: /home/arthur/Téléchargements/test/20250208/AC__TC__2B/
Created local directory: /home/arthur/Téléchargements/test/20250209/AC__TC__2B/
Created local directory: /home/arthur/Téléchargements/test/20250210/AC__TC__2B/
Created local directory: /home/arthur/Téléchargements/test/20250211/AC__TC__2B/
Created local directory: /home/arthur/Téléchargements/test/20250212/AC__TC__2B/
Created local directory: /home/arthur/Téléchargements/test/20250213/AC__TC__2B/
Created local directory: /home/arthur/Téléchargements/test/20250214/AC__TC__2B/
Created local directory: /home/arthur/Téléchargements/test/20250215/AC__TC__2B/
Created local directory: /home/arthur/Téléchargements/test/20250216/AC__TC__2B/
Created local directory: /home/arthur/Té

In [ ]:
# Download individual files

local_dir = "/home/bpiskala/Object_Data/EarthCARE/h5/20250116/MSI_COP_2A/"
os.makedirs(local_dir, exist_ok=True)

files_to_download = [
    "ECA_EXAB_MSI_COP_2A_20250116T034547Z_20250116T072939Z_03613B.ZIP",
    "ECA_EXAB_MSI_COP_2A_20250116T192302Z_20250116T212825Z_03623C.ZIP"
]

lftp_script = f"""
open -u {username},{password} ftps://{host}
set ssl:verify-certificate no
set ftp:ssl-auth TLS
set ftp:passive-mode true
lcd {local_dir}
"""

for file_name in files_to_download:
    remote_path = f"/EarthCARE/EarthCAREL2Products/MSI_COP_2A/AB/2025/01/16/{file_name}"
    lftp_script += f"get {remote_path} -o {file_name}\n"

lftp_script += "quit\n"

# Execute lftp command
try:
    process = subprocess.run(
        ["lftp", "-c", lftp_script],
        capture_output=True,
        text=True
    )

    print("STDOUT:", process.stdout)
    print("STDERR:", process.stderr)

    if process.returncode == 0:
        print("FTPS file transfer completed successfully")
    else:
        print("FTPS transfer failed")

except Exception as e:
    print("Error executing lftp command:", e)

STDOUT: 
STDERR: 
FTPS file transfer completed successfully


In [ ]:
# Extract h5 files and delete ZIPs
local_dir = "/home/bpiskala/Object_Data/EarthCARE/h5/20250116/MSI_COP_2A/"

for filename in os.listdir(local_dir):
    if filename.endswith(".ZIP"):
        zip_path = os.path.join(local_dir, filename)
        
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            for file in zip_ref.namelist():
                if file.endswith(".h5"):
                    zip_ref.extract(file, local_dir)

        os.remove(zip_path)
        print(f"Processed and deleted: {zip_path}")

print("Extraction complete.")

Processed and deleted: /home/bpiskala/Object_Data/EarthCARE/h5/20250116/MSI_COP_2A/ECA_EXAB_MSI_COP_2A_20250116T034547Z_20250116T072939Z_03613B.ZIP
Processed and deleted: /home/bpiskala/Object_Data/EarthCARE/h5/20250116/MSI_COP_2A/ECA_EXAB_MSI_COP_2A_20250116T192302Z_20250116T212825Z_03623C.ZIP
Extraction complete.
